In [1]:
!which python

/Users/mesa/Coding Projects/LLM-Debug-Assistant/venv/bin/python


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

/Users/mesa/Coding Projects/LLM-Debug-Assistant/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/mesa/Coding Projects/LLM-Debug-Assistant/venv/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [3]:
# Model from Hugging Face hub
base_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# New instruction dataset
stack_overflow_dataset = "Mxode/StackOverflow-QA-C-Language-40k"
# other datasets


# Fine-tuned model
new_model = "CustomLLM"

In [4]:
dataset = load_dataset(stack_overflow_dataset, split="train")

In [5]:
# dataset currently has 'question' and 'answer' columns. We need to combine them into a single column named 'text'
dataset = dataset.map(
    lambda examples: {
        "text": "<s>[INST]" + examples["question"] + "[\INST] " + examples["answer"] + "</s>",
    }
)

In [6]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    # quantization_config=quant_config,
    # device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [8]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [9]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [10]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    # optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [11]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/Users/mesa/Coding Projects/LLM-Debug-Assistant/venv/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/Users/mesa/Coding Projects/LLM-Debug-Assistant/venv/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [60]:
trainer.train()

Step,Training Loss
25,0.918700
50,0.000000
75,0.000000
100,0.000000
125,0.000000
150,0.000000
175,0.000000
200,0.000000
225,0.000000
250,0.000000


TrainOutput(global_step=10163, training_loss=0.002259835781197631, metrics={'train_runtime': 7860.1172, 'train_samples_per_second': 5.172, 'train_steps_per_second': 1.293, 'total_flos': 5.700391792559309e+16, 'train_loss': 0.002259835781197631, 'epoch': 1.0})

In [62]:
trainer.save_model(new_model)
trainer.tokenizer.save_pretrained(new_model)

('CustomLLM/tokenizer_config.json',
 'CustomLLM/special_tokens_map.json',
 'CustomLLM/tokenizer.json')

Now lets compare asking 3 questions to our fine tuned model vs our base model

## fine tuned model

In [15]:
#logging.set_verbosity(logging.CRITICAL)

prompt = "How do I loop through an array in C?"
pipe = pipeline(task="text-generation", model=new_model, tokenizer=tokenizer, max_length=500)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] How do I loop through an array in C? [/INST]

I'm not able to provide you with a specific example, but I can give you an overview of how to loop through an array in C.

1. Declare an array:

```c
int arr[5] = {1, 2, 3, 4, 5};
```

2. Initialize the array:

```c
int i;
for (i = 0; i < 5; i++) {
    printf("%d ", arr[i]);
}
```

3. Print the array:

```c
printf("\n");
for (i = 0; i < 5; i++) {
    printf("%d ", arr[i]);
}
```

4. Add a new element to the array:

```c
arr[4] = 6;
```

5. Print the updated array:

```c
printf("\n");
for (i = 0; i < 5; i++) {
    printf("%d ", arr[i]);
}
```

6. Remove an element from the array:

```c
arr[3] = 7;
```

7. Print the updated array:

```c
printf("\n");
for (i = 0; i < 5; i++) {
    printf("%d ", arr[i]);
}
```

I hope this helps! Let me know if you have any further questions.


In [4]:
prompt = "How do I loop through an array in C?"
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=500)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

NameError: name 'base_model' is not defined

# Evaluation

In [12]:
from tensorboard import notebook
log_dir = "results/runs"
notebook.start("--logdir {} --port 4000".format(log_dir))

Reusing TensorBoard on port 4000 (pid 61423), started 15:01:37 ago. (Use '!kill 61423' to kill it.)

In [15]:
%load_ext tensorboard

%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
